In [ ]:
import torch
import torch.nn as nn
class Mynetwork(nn.Module):
    def __init__(self,input_num=1 , out_num=1,hidden_num=32):
        super().__init__()
        self.MLP=nn.Sequential(
            nn.Linear(input_num, hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,out_num),
        )
        pass
    def forward(self,x):
        return self.MLP(x)

In [ ]:
A=torch.tensor([[3/2,-1/2],[-1/2,3/2]])

eigenvalues= torch.linalg.eigvalsh(A)
print(eigenvalues)

In [ ]:
dtype=torch.float32
NN=Mynetwork(1,1).to(dtype=dtype)
input=torch.tensor([[1.0],[2.0]],dtype=dtype)
print(NN(input))

In [ ]:
output=NN(input)
diag=torch.diag(output.flatten())
B=torch.tensor([[0,-1/2],[-1/2,0]],dtype=torch.float32)+diag
print(B)

In [ ]:
pre_eig=torch.linalg.eigvalsh(B)
print(pre_eig)

In [ ]:
optimizer=torch.optim.Adam(NN.parameters(),lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=20,threshold=1e-4)
loss_fn=nn.L1Loss()
for i in range(1000):
    optimizer.zero_grad()
    
    output=NN(input)
    diag=torch.diag(output.flatten())
    B=torch.tensor([[0,-1/2],[-1/2,0]],dtype=torch.float32)+diag
    pre_eig=torch.linalg.eigvalsh(B)
    
    loss=loss_fn(eigenvalues[0],pre_eig[0])
    loss.backward()
    # print(NN.MLP[0].weight.grad)
    optimizer.step()
    scheduler.step(loss)
    if i %10==0:
        print(pre_eig)
        print(loss.item())